<a href="https://colab.research.google.com/github/manjitullal/foursquare/blob/master/FourSquare_Temporal_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Temporal and Spatial analysis of events data using LSTM**



**Dataset:** foursquare

**Aim:** to predict future location and time of user given the historical sequences of location and time .

An analogy for the aim is , predicting the next word in a sentence. 


**Contents:**
***
1. Data pre processing
2. Encoding
3. Modeling
4. Training

# **1. Data pre processing**


In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf


from google.colab import drive
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torch.nn.utils.rnn import pad_sequence

torch.manual_seed(1)

1. **Check for GPU**
---
This code presently does not use GPU. This will later be modified to use one.


In [2]:
#check the devices available

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16114837832282851235
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11735902249367462014
physical_device_desc: "device: XLA_CPU device"
]


In [3]:
#check if the gpu is available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [4]:
#the GPU may not be available at the moment

import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU device not found')


GPU device not found


In [5]:
#dataset is in the google drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#my google drive path

!ls "/content/drive/My Drive/dataset/foursquare"
path = "/content/drive/My Drive/dataset/foursquare"

Checkin.txt	 UserFriends.txt    VenueRating.txt
Description.txt  VenueCategory.txt  Venue.txt


2. **Load dataset.** 
---
Dataset, consists of 5 tables. `Checkin` table has the event details

In [7]:
%%time
Checkin_columns = ['UserID','VenueID','Year','Month','Date','Hour']
Checkin = pd.read_csv(path+'/Checkin.txt', sep=',', skiprows=1, names=Checkin_columns)

CPU times: user 591 ms, sys: 200 ms, total: 790 ms
Wall time: 902 ms


In [8]:
%%time

Venue_columns = ['VenueID','VenueName','Latitude','Longitude','CategoryID']
Venue = pd.read_csv(path+'/Venue.txt', sep=',', error_bad_lines=False,skiprows=1,names=Venue_columns)

VenueCategory_columns = ['CategoryID','CategoryName','ParentCategoryID']
VenueCategory = pd.read_csv(path+'/VenueCategory.txt', sep=',',error_bad_lines=False,skiprows=1,names=VenueCategory_columns)

VenueRating_columns = ['VenueID','Rating']
VenueRating = pd.read_csv(path+'/VenueRating.txt', sep=',',error_bad_lines=False,skiprows=1,names=VenueRating_columns)

UserFriends = pd.read_csv(path+'/UserFriends.txt', sep=',') 

all_tables = [Checkin,Venue,VenueCategory,VenueRating,UserFriends]
all_tables_string = ['Checkin','Venue','VenueCategory','VenueRating','UserFriends']

CPU times: user 811 ms, sys: 106 ms, total: 916 ms
Wall time: 1.04 s


In [9]:
Checkin.head()

,UserID,VenueID,Year,Month,Date,Hour
0,u1302,v47,2012,2,24,11
1,u45,v132,2012,2,24,11
2,u24844,v86,2012,2,24,11
3,u896,v248,2012,2,24,11
4,u5020,v29,2012,2,24,11


In [10]:
#stats of the data 

def _describe(data):
    print(f" Number of rows: {data.shape[0]}")
    print(f" Number of columns: {data.shape[1]}")
    print(f" Number of null values: {np.sum(data.isnull().sum())}")
    print("The columns that have null values")
    print(pd.DataFrame(data.isnull().sum()).T)
    
for index,table in enumerate(all_tables):
    print(f"Details of table {all_tables_string[index]}")
    print("")
    _describe(table)
    print("")

Details of table Checkin

 Number of rows: 1276988
 Number of columns: 6
 Number of null values: 0
The columns that have null values
   UserID  VenueID  Year  Month  Date  Hour
0       0        0     0      0     0     0

Details of table Venue

 Number of rows: 85928
 Number of columns: 5
 Number of null values: 14
The columns that have null values
   VenueID  VenueName  Latitude  Longitude  CategoryID
0        0         12         2          0           0

Details of table VenueCategory

 Number of rows: 394
 Number of columns: 3
 Number of null values: 0
The columns that have null values
   CategoryID  CategoryName  ParentCategoryID
0           0             0                 0

Details of table VenueRating

 Number of rows: 68178
 Number of columns: 2
 Number of null values: 96
The columns that have null values
   VenueID  Rating
0       96       0

Details of table UserFriends

 Number of rows: 1366388
 Number of columns: 2
 Number of null values: 0
The columns that have null valu

3. **Model for one user** (for testing), hence we will filter the data for one user, eventually this will be extended for all users

In [11]:
#filter data for one user 

Checkin_u1205 = Checkin[Checkin.UserID == 'u1205']
Checkin_u1205.head()

,UserID,VenueID,Year,Month,Date,Hour
2723,u1205,v73805,2012,2,25,9
3817,u1205,v9884,2012,2,25,11
4739,u1205,v3906,2012,2,25,13
5904,u1205,v10373,2012,2,25,15
6840,u1205,v9884,2012,2,25,17


In [12]:
# drop userid as that is not useful now, since there is only one user 

Checkin_u1205.drop(['UserID'], axis=1, inplace=True)

#renaming column Date to Day
Checkin_u1205.rename(columns={"Date":"Day"}, inplace=True)
Checkin_u1205.head()

,VenueID,Year,Month,Day,Hour
2723,v73805,2012,2,25,9
3817,v9884,2012,2,25,11
4739,v3906,2012,2,25,13
5904,v10373,2012,2,25,15
6840,v9884,2012,2,25,17


In [13]:
# create a new column, datetime to sort the events 

%%time
Checkin_u1205['Datetime'] = pd.to_datetime(Checkin_u1205[['Year', 'Month', 'Day', 'Hour']])
Checkin_u1205.head()

CPU times: user 17.1 ms, sys: 1.03 ms, total: 18.2 ms
Wall time: 20.8 ms


In [14]:
# sort based on datetime
Checkin_u1205.sort_values(by='Datetime',inplace=True)

Checkin_u1205.head()

,VenueID,Year,Month,Day,Hour,Datetime
2723,v73805,2012,2,25,9,2012-02-25 09:00:00
9154,v40561,2012,2,25,9,2012-02-25 09:00:00
3817,v9884,2012,2,25,11,2012-02-25 11:00:00
10664,v1743,2012,2,25,11,2012-02-25 11:00:00
4739,v3906,2012,2,25,13,2012-02-25 13:00:00


In [15]:
# from the above, we can see that for some reason there are 2 duplicate timestamps with different venues
# it is not possible for a person to be at different location at the same time, so removing the rows with duplicate time stamps
# the category of the venues is hierarchical, however for there appears no link between the venues

Checkin_u1205_nodup = Checkin_u1205.drop_duplicates('Datetime')

In [16]:
print("Rows in Checkin_u1205: ", Checkin_u1205.shape[0])
print("Rows in Checkin_u1205_nodup: ", Checkin_u1205_nodup.shape[0])

Rows in Checkin_u1205:  1303
Rows in Checkin_u1205_nodup:  1227


In [17]:
Checkin_u1205_nodup.head()

,VenueID,Year,Month,Day,Hour,Datetime
2723,v73805,2012,2,25,9,2012-02-25 09:00:00
3817,v9884,2012,2,25,11,2012-02-25 11:00:00
4739,v3906,2012,2,25,13,2012-02-25 13:00:00
5904,v10373,2012,2,25,15,2012-02-25 15:00:00
6840,v9884,2012,2,25,17,2012-02-25 17:00:00


In [18]:
Checkin_u1205_nodup.iloc[:20]

,VenueID,Year,Month,Day,Hour,Datetime
2723,v73805,2012,2,25,9,2012-02-25 09:00:00
3817,v9884,2012,2,25,11,2012-02-25 11:00:00
4739,v3906,2012,2,25,13,2012-02-25 13:00:00
5904,v10373,2012,2,25,15,2012-02-25 15:00:00
6840,v9884,2012,2,25,17,2012-02-25 17:00:00
18507,v9885,2012,2,26,9,2012-02-26 09:00:00
12801,v10373,2012,2,26,11,2012-02-26 11:00:00
11674,v9885,2012,2,26,13,2012-02-26 13:00:00
15263,v2927,2012,2,26,15,2012-02-26 15:00:00
20745,v6013,2012,2,27,9,2012-02-27 09:00:00


For time being we are not using the heirarchical informations about the venue, to keep the baseline model simple.

Now, we need to create a time-series of events. 

`Example:`

User goes to gym, grocery and home in that order or shopping, movies, restaurant and home in that order. 

Here, we need to create the longest time-series.


4. **Create longest sequences.** 
---
`Idea`: gather longest time-series by viewing the events of the user, events less than the duration of 8hrs between them will be added to the same series. A gap of 8 hrs or more indicates the day has ended for the user, hence no more travel.


In [19]:

%%time

import datetime

def _generate_events(data):
  previous_time = datetime.datetime(2020, 12, 31)
  all_events = []
  current_events = []
  for index, row in data.iterrows():
    current_time = row['Datetime']  
    current_hour = row['Hour']
    venue = row['VenueID']
    if( (current_time - previous_time).total_seconds()/60/60 < 8):
      current_events.append([venue, current_hour])
      previous_time = current_time
    else:
      all_events.append(current_events)
      current_events = []
      current_events.append([venue, current_hour])
      previous_time = current_time
  if len(current_events)>0:
    all_events.append(current_events)
  return all_events



CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 7.63 µs


5. **Subsequences:** A Sequence cannot be directly fed into a model, hence we need to break it down to components.

---

Example:

for a sequence (v1,t1) (v2,t2) (v3,t3) (v4,t4)

(v1,t1) (v2,t2)

(v2,t2) (v3,t3)

(v3,t3) (v4,t4)


In [20]:
# create all possible subsequences from the above sequence (maintaining the order)
# we need a length of atleast 2 for feature and the label, ignore all sequence of events less than length 2


# LSTM will remember that v2 comes after v1 and that v3 comes after v2 and if it sees v3 then next would be v4

%%time

def _generate_subsequence(data):
  all_sequences = []
  for sequence in data:  
    if len(sequence) >= 2:
      sequences = []
      for i in range(0,len(sequence)-1):
          sequences.append(sequence[i:i+2])          
      all_sequences.append(sequences)
  return all_sequences


#old way of creating sub sequences 
# (v1,t1) (v2,t2)
# (v1,t1) (v2,t2) (v3,t3)
# (v1,t1) (v2,t2) (v3,t3) (v4,t4)

'''
def _generate_subsequence(data):
  all_sequences = []
  for sequence in data:  
    if len(sequence) > 2:
      for i in range(0,1):
        sequences = []
        for j in range(i+2,len(sequence)+1):      
          sequences.append(sequence[i:j])          
        all_sequences.append(sequences)
  return all_sequences
'''


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.53 µs


# **2. Encoding**


In [21]:
%time

events = _generate_events(Checkin_u1205_nodup)
#print("Events")
#print(events)

sequences = _generate_subsequence(events)
#print("Subsequences")
#print(sequences)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


1. **Split the data:** 

---
Before, encoding, split the data to train, validation and test set and then perform the encoding and padding. We do this to maintain integrity and structure of the data.

Since we have inputs of the different length, we will have to pad each input with zeros , so that all inputs are of the same length in a mini-batch

60, 20, 20 split (train, validation and test)

In [22]:
n = len(events)  

n_test = int( n * .2 ) 
n_val = int( n * .2 ) 
n_train = n - (n_test+n_val)

print(f"Dataset set terms: {n}")
print(f"Train set terms: {n_train}") 
print(f"Test set terms: {n_test}")
print( f"Validation set terms: {n_val}")

train_set, val_set, test_set = data.random_split(events, (n_train, n_val, n_test))

#the train, val and the test indices has to be whole numbers adding upto to the n

Dataset set terms: 154
Train set terms: 94
Test set terms: 30
Validation set terms: 30


2. **Get Max length of each set.**

---
we need this info for padding.

In [23]:
#find max length of sequence for each set

def _get_maxlength_seq(data):
  t = []
  for i in data:
    t.append([i,len(i)])
  temp_df = pd.DataFrame(t)
  temp_df.sort_values(by=1, ascending=False,inplace=True)
  return np.max(temp_df[1])

max_length_train= _get_maxlength_seq(train_set)
max_length_val= _get_maxlength_seq(val_set)
max_length_test= _get_maxlength_seq(test_set)

print(f"max_length_train: {max_length_train}")
print(f"max_length_val: {max_length_val}")
print(f"max_length_test: {max_length_test}")

max_length_train: 14
max_length_val: 18
max_length_test: 22


3. **One hot ecoding** of the the venues then do padding.

----

Pytorch does not have one hot encoding package hence, need to create our own code.

In [24]:

class one_hot_char_coding(object):
  def __init__(self):
    self.word2idx = {}
    self.idx2word = []
    self.length = 0

  def add_word(self, word):
    if word not in self.idx2word:
      self.idx2word.append(word)
      self.word2idx[word] = self.length + 1
      self.length += 1
    return self.word2idx[word]

  def __len__(self):
    return len(self.idx2word)

  def onehot_encoded(self, word):
    vec = np.zeros(self.length)
    vec[self.word2idx[word]-1] = 1
    return vec


In [25]:
coded_obj = one_hot_char_coding()

for word in Checkin_u1205_nodup.VenueID.values:
    coded_obj.add_word(word)
#print(coded_obj.word2idx)
#print(coded_obj.onehot_encoded("v45783"))
#here we have 139 unique values, so the one hot encoded vector is 139 length long

4. **Coding time:**
---
Time is cyclical feature so it will be encoded as Time Column = Sin(Time) + Cosine(Time). This will maintain the cyclic nature of the data.

sequence with less than 2 will be descarded as they cannot be used to predict


In [26]:
%time

def _assign_code(data):
  temp_row=[]
  for row in data:    
    temp_event = []
    for event in row:   
      venue_id = coded_obj.onehot_encoded(event[0])
      time_sine = np.sin(2 * np.pi * event[1]/23.0)
      time_cos = np.cos(2 * np.pi * event[1]/23.0)
      temp_event.append([venue_id, time_sine, time_cos])
    if(len(temp_event) >= 2):
      temp_row.append(temp_event)
  return temp_row
 
train_set_e = _assign_code(train_set)
print(f"Number of inputs less than 2 is: {len(train_set) - len(train_set_e)}") 


CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.44 µs
Number of inputs less than 2 is: 8


5. **Features and Labels**

---
Last event will be the target 

In [27]:
def _separate_x_y(data):
  for row in data:
    x = row[:-1]
    y = row[-1]
    return [x,y]

#_separate_x_y(train_set_e)

6. **Padding**

Pad with zeros to even the length of the input

In [28]:
'''
a = torch.ones(1,3)
b = torch.ones(1,5)
c = torch.ones(1,6)
F.pad(a,(0,3), "constant",0)'''


'\na = torch.ones(1,3)\nb = torch.ones(1,5)\nc = torch.ones(1,6)\nF.pad(a,(0,3), "constant",0)'

Resources used
***
https://discuss.pytorch.org/t/mini-batch-training-for-inputs-of-variable-sizes/13662

https://medium.com/ibm-data-science-experience/markdown-for-jupyter-notebooks-cheatsheet-386c05aeebed

https://stackoverflow.com/questions/53532352/how-do-i-split-the-training-dataset-into-training-validation-and-test-datasets

https://learning.oreilly.com/library/view/deep-learning-with/9781788624336/e397c9a2-28a2-41fd-bcc2-a6bb49adfc44.xhtml

https://www.kaggle.com/avanwyk/encoding-cyclical-features-for-deep-learning
